<a href="https://colab.research.google.com/github/Lavya-Thapar/Spam-detection/blob/main/Topic_Modelling(LDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim.models.coherencemodel import CoherenceModel
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
data = pd.read_csv("training_articles_csv_file.csv", encoding = 'latin1')

print(data.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


                                             article  \
0  Engineering societies provide invaluable oppor...   
1  Active participation in engineering societies ...   
2  Data structures and algorithms (DSA) form the ...   
3  In addition to technical expertise, soft skill...   
4  Balancing academics and extracurricular activi...   

                                               topic  \
0  The Role of Engineering Societies in Skill Dev...   
1  Importance of Active Participation in Engineer...   
2  Leveraging Data Structures and Algorithms (DSA...   
3                             Developing Soft Skills   
4  Strategies for Enhancing CGPA,Balancing Academ...   

                                                tags  
0  Engineering Societies, Skill Development, Netw...  
1  Active Participation, Holistic Growth, Enginee...  
2  Data Structures, Algorithms, Efficient Problem...  
3                   Soft Skills, Development, Skills  
4               Enhancing CGPA, Balancing, Academic

In [ ]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    # Join tokens to form preprocessed text
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

data['article'] = data['article'].apply(preprocess_text)
print(data['article'])

0     engineering society provide invaluable opportu...
1     active participation engineering society cruci...
2     data structure algorithm dsa form backbone com...
3     addition technical expertise soft skill essent...
4     balancing academic extracurricular activity es...
5     effective interview preparation essential land...
6     engineering graduate diverse career opportunit...
7     artificial intelligence ai revolutionizing soc...
8     ethic responsibility paramount development dep...
9     ai machine learning offer vast career opportun...
10    future work shaped automation ai technology ad...
11    integrating technical nontechnical skill essen...
12    communication skill indispensable engineering ...
13    leadership development essential engineering p...
14    building strong personal brand essential engin...
15    design thinking increasingly integrated engine...
16    open source contribution offer valuable opport...
17    cybersecurity awareness essential engineer

In [ ]:
x_prev=data['article']
vectorizer =TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 2))
x = vectorizer.fit_transform(data['article'].values)


In [ ]:
# def compute_coherence_score(lda_model, texts, dictionary, coherence='c_v'):
#     coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence=coherence)
#     coherence_score = coherence_model.get_coherence()
#     return coherence_score

In [ ]:
# def choosing_best_ncomponent(train_data):
#   n_components_range = [5, 10, 15, 20, 25]

#   best_n_components = None
#   best_coherence_score = -1  # Initialize with a low value
#   train_data=train_data.apply(preprocess_text)
# # Iterate over each value of n_components
#   for n_components in n_components_range:
#       # Train LDA model
#       lda = LatentDirichletAllocation(n_components=n_components, random_state=42)
#       lda.fit(train_data)

#       # Convert training data to a list of tokenized documents
#       tokenized_texts = [doc.split() for doc in train_data]

#       # Create a dictionary from tokenized documents
#       dictionary = gensim.corpora.Dictionary(tokenized_texts)

#       # Compute coherence score
#       coherence = compute_coherence_score(lda, tokenized_texts, dictionary)
#       print(f"n_components={n_components}: Coherence Score={coherence}")

#       # Update best n_components if coherence score improves
#       if coherence > best_coherence_score:
#           best_coherence_score = coherence
#           best_n_components = n_components

#       # Print the best value of n_components
#       print(f"Best value of n_components: {best_n_components} (Coherence Score={best_coherence_score})")
#       return best_n_components

In [ ]:
lda = LatentDirichletAllocation(n_components=48, learning_method='online', random_state=42)
lda.fit(x)

topic_modelling = lda.transform(x)

topic_labels = np.argmax(topic_modelling, axis=1)
data['topic_labels'] = topic_labels
print(data)

                                              article  \
0   engineering society provide invaluable opportu...   
1   active participation engineering society cruci...   
2   data structure algorithm dsa form backbone com...   
3   addition technical expertise soft skill essent...   
4   balancing academic extracurricular activity es...   
5   effective interview preparation essential land...   
6   engineering graduate diverse career opportunit...   
7   artificial intelligence ai revolutionizing soc...   
8   ethic responsibility paramount development dep...   
9   ai machine learning offer vast career opportun...   
10  future work shaped automation ai technology ad...   
11  integrating technical nontechnical skill essen...   
12  communication skill indispensable engineering ...   
13  leadership development essential engineering p...   
14  building strong personal brand essential engin...   
15  design thinking increasingly integrated engine...   
16  open source contribution of

In [ ]:
def extract_keywords(sentence):
    # Tokenize the sentence into words
    words = word_tokenize(sentence)

    # Perform part-of-speech tagging to identify nouns and adjectives
    tagged_words = pos_tag(words)

    # Extract nouns and adjectives as keywords
    keywords = [word for word, tag in tagged_words if tag.startswith('NN') or tag.startswith('JJ')]

    return keywords

In [ ]:
new_article_text = "Take part in as many hackathons as possible"

# Preprocess the text
preprocessed_new_article = preprocess_text(new_article_text)

# Transform the preprocessed text into vector space
tfidf_new_article = vectorizer.transform([preprocessed_new_article])

# Transform the vectorized text into topic space
topic_modelling_new_article = lda.transform(tfidf_new_article)

# Find the dominant topic for the new article
dominant_topic_index = np.argmax(topic_modelling_new_article)
dominant_topic = dominant_topic_index  # Topic indices are zero-based
filtered_rows = data[data['topic_labels'] == dominant_topic]

# Printing the value of the first attribute from the filtered rows
if not filtered_rows.empty:
    print("Suggested tags for your article are:", filtered_rows['topic'])
    for sentence in filtered_rows['topic']:
      print(extract_keywords(sentence))
else:
    print("No matching rows found.")
# print("Dominant Topic for the New Article:", dominant_topic)

Suggested tags for your article are: 2     Leveraging Data Structures and Algorithms (DSA...
15    The Intersection of Design Thinking and Engine...
33    Leveraging Hackathons for Innovation and Colla...
41    The Role of Engineering in Space Exploration a...
43    The Importance of Cross-Disciplinary Collabora...
Name: topic, dtype: object
['Data', 'Structures', 'Algorithms', 'DSA', 'Efficient', 'Problem']
['Intersection', 'Design', 'Thinking', 'Engineering', 'Innovation']
['Hackathons', 'Innovation', 'Collaboration', 'Tech', 'Communities']
['Role', 'Engineering', 'Space', 'Exploration', 'Colonization']
['Importance', 'Cross-Disciplinary', 'Collaboration', 'Complex', 'Problems']
